# Core 

> Minimal pipeline for Diffusion components.

In [ ]:
#| default_exp core

In [ ]:
#| export
from types import SimpleNamespace
from fastcore.basics import store_attr
# imports for diffusion models
import torch
from tqdm.auto import tqdm 
from transformers import logging
from transformers import CLIPTextModel, CLIPTokenizer
from huggingface_hub import notebook_login
from diffusers import StableDiffusionPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers import LMSDiscreteScheduler

In [ ]:
#| export
# bring in all the helper function to process and plot latents
from min_diffusion.utils import text_embeddings, image_from_latents

In [ ]:
#| export

class MinimalDiffusion:
    def __init__(self, model_name, device, dtype):
        store_attr()

    def load_sd_pieces(
        self,
        attention_slicing=False,
        better_vae=''):
        "Loads and returns the individual pieces in a Diffusion pipeline."
        model_name = self.model_name
        dtype = self.dtype
        
        # create the tokenizer and text encoder
        tokenizer = CLIPTokenizer.from_pretrained(
            model_name,
            subfolder="tokenizer",
            torch_dtype=dtype)
        text_encoder = CLIPTextModel.from_pretrained(
            model_name,
            subfolder="text_encoder",
            torch_dtype=dtype).to(self.device)

        # we are using a VAE from stability that was trained for longer than the baseline 
        if better_vae:
            assert better_vae in ('ema', 'mse')
            vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{better_vae}", torch_dtype=dtype).to(self.device)
        else:
            vae = AutoencoderKL.from_pretrained(model_name, subfolder='vae', torch_dtype=dtype).to(self.device)
        
        # build the unet
        unet = UNet2DConditionModel.from_pretrained(
            model_name,
            subfolder="unet",
            torch_dtype=dtype).to(self.device)
        
        # enable unet attention slicing
        if attention_slicing:
            print('Enabling default unet attention slicing.')
            slice_size = unet.config.attention_head_dim // 2
            unet.set_attention_slice(slice_size)
            
        # build the scheduler
        scheduler = LMSDiscreteScheduler.from_config(model_name, subfolder="scheduler")
        
        # create and return a simple pipeline with all of the pieces
        self.tokenizer = tokenizer
        self.text_encoder = text_encoder
        self.vae = vae
        self.unet = unet
        self.scheduler = scheduler

    def set_init_latents(self, latents):
        self.init_latents = latents

    def generate(
        self,
        prompt,
        guide_tfm=None,
        width=512,
        height=512,
        steps=50,
        **kwargs):
        # make sure we have a guidance transformation
        assert guide_tfm
        
        # prepare the text embeddings
        text = text_embeddings(prompt, self.tokenizer, self.text_encoder, device=self.device)
        uncond = text_embeddings('', self.tokenizer, self.text_encoder, device=self.device)
        emb = torch.cat([uncond, text]).type(self.unet.dtype)
        
        # start from the shared, initial latents
        latents = torch.clone(self.init_latents)
        self.scheduler.set_timesteps(steps)
        latents = latents * self.scheduler.init_noise_sigma
        
        # run the diffusion process
        for i,ts in enumerate(tqdm(self.scheduler.timesteps)):
            inp = self.scheduler.scale_model_input(torch.cat([latents] * 2), ts)
            with torch.no_grad(): 
                tf = ts
                if torch.has_mps:
                    tf = ts.type(torch.float32)
                u,t = self.unet(inp, tf, encoder_hidden_states=emb).sample.chunk(2)
            
            # call the guidance transform
            pred = guide_tfm(u, t, idx=i)
            
            # update the latents
            latents = self.scheduler.step(pred, ts, latents).prev_sample
            
        # decode the final latents and return the generated image
        image = image_from_latents(latents, self.vae)
        return image    


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()